In [10]:
import numpy as np
import pandas as pd
import warnings


In [11]:
warnings.filterwarnings('ignore')

In [12]:
colmname=['user_id','Item_id','Rating','Timestamp']
df=pd.read_csv('u.data',sep="\t",names=colmname)

In [13]:
df.head()

,user_id,Item_id,Rating,Timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [16]:
df.shape

(100000, 4)

In [17]:
df['user_id'].nunique()

943

In [18]:
-df['Item_id'].nunique()

-1682

In [19]:
movies_title=pd.read_csv('u.item',encoding="cp1252",sep="\|",header=None)

In [20]:
movies_title.shape

(1682, 24)

In [21]:
movies_titles=movies_title[[0,1]]
movies_titles.columns=["Item_id","Title"]
movies_titles.head()

,Item_id,Title
0,1,Toy Story (1995)
1,2,GoldenEye (1995)
2,3,Four Rooms (1995)
3,4,Get Shorty (1995)
4,5,Copycat (1995)


In [22]:
df=pd.merge(df,movies_titles,on="Item_id")
df.head()

,user_id,Item_id,Rating,Timestamp,Title
0,196,242,3,881250949,Kolya (1996)
1,63,242,3,875747190,Kolya (1996)
2,226,242,5,883888671,Kolya (1996)
3,154,242,3,879138235,Kolya (1996)
4,306,242,5,876503793,Kolya (1996)


In [23]:
df.tail()

,user_id,Item_id,Rating,Timestamp,Title
99995,840,1674,4,891211682,Mamma Roma (1962)
99996,655,1640,3,888474646,"Eighth Day, The (1996)"
99997,655,1637,3,888984255,Girls Town (1996)
99998,655,1630,3,887428735,"Silence of the Palace, The (Saimt el Qusur) (1..."
99999,655,1641,3,887427810,Dadetown (1995)


In [24]:
ratings=pd.DataFrame(df.groupby('Title').mean()['Rating'])

In [25]:
ratings.head()

,Rating
Title,
'Til There Was You (1997),2.333333
1-900 (1994),2.600000
101 Dalmatians (1996),2.908257
12 Angry Men (1957),4.344000
187 (1997),3.024390


In [26]:
ratings['num of ratings']=pd.DataFrame(df.groupby('Title').count()['Rating'])
ratings.head()

,Rating,num of ratings
Title,,
'Til There Was You (1997),2.333333,9
1-900 (1994),2.600000,5
101 Dalmatians (1996),2.908257,109
12 Angry Men (1957),4.344000,125
187 (1997),3.024390,41


In [27]:
moviemat=df.pivot_table(index="user_id",columns="Title",values="Rating")
movies_title.head()

,0,1,2,3,4,5,6,7,8,9,...,14,15,16,17,18,19,20,21,22,23
0,1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
1,2,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,...,0,0,0,0,0,0,0,1,0,0
2,3,Four Rooms (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Four%20Rooms%...,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,4,Get Shorty (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Get%20Shorty%...,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,Copycat (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Copycat%20(1995),0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [28]:
movies_title=movies_title[[0,1]]
movies_title.columns=["Item_id","Title"]
movies_title.head()

,Item_id,Title
0,1,Toy Story (1995)
1,2,GoldenEye (1995)
2,3,Four Rooms (1995)
3,4,Get Shorty (1995)
4,5,Copycat (1995)


In [29]:
def predict_movies(movie_name):
    movie_user_ratings=moviemat[movie_name]
    similar_to_movie=moviemat.corrwith(movie_user_ratings)
    corr_movie=pd.DataFrame(similar_to_movie,columns=['correlation'])
    corr_movie.dropna(inplace=True)
    corr_movie=corr_movie.join(ratings['num of ratings'])
    predictions=corr_movie[corr_movie['num of ratings']>100].sort_values('correlation',ascending=False)
    return predictions

In [35]:
predict_my_movie=predict_movies("GoldenEye (1995)")

In [36]:
predict_my_movie.head()

,correlation,num of ratings
Title,,
GoldenEye (1995),1.000000,131
Ulee's Gold (1997),0.806091,184
Down Periscope (1996),0.728723,101
Jungle2Jungle (1997),0.646619,132
"Philadelphia Story, The (1940)",0.600751,104
